In [ ]:
%run "Helper_Scripts/CWLite_Connect.ipynb"

In [ ]:
%run "Helper_Scripts/Setup_Target_Generic.ipynb"

In [ ]:
# uncomment based on your target
#%run "Helper_Scripts/Program_XMEGA.ipynb"
%run "Helper_Scripts/Program_STM.ipynb"
#%run "Helper_Scripts/No_Programmer.ipynb"
fw_path = "../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-cwlitearm.hex"

In [ ]:
# program the target
program_target(scope, fw_path)

In [ ]:
#Capture Traces
from tqdm import tqdm
from chipwhisperer.capture.acq_patterns.basic import AcqKeyTextPattern_Basic
import numpy as np
import time

ktp = AcqKeyTextPattern_Basic(target=target)

traces = []
textin = []
keys = []
N = 150  # Number of traces
target.init()
for i in tqdm(range(N), desc='Capturing traces'):
    # run aux stuff that should come before trace here

    key, text = ktp.newPair()  # manual creation of a key, text pair can be substituted here
    textin.append(text)
    keys.append(key)

    #target.reinit()

    target.setModeEncrypt()  # only does something for targets that support it
    target.loadEncryptionKey(key)
    target.loadInput(text)

    # run aux stuff that should run before the scope arms here

    scope.arm()

    # run aux stuff that should run after the scope arms here

    target.go()
    timeout = 50
    # wait for target to finish
    while target.isDone() is False and timeout:
        timeout -= 1
        time.sleep(0.01)

    try:
        ret = scope.capture()
        if ret:
            print('Timeout happened during acquisition')
    except IOError as e:
        print('IOError: %s' % str(e))

    # run aux stuff that should happen after trace here
    _ = target.readOutput()  # clears the response from the serial port
    traces.append(scope.getLastTrace())

#Convert traces to numpy arrays
trace_array = np.asarray(traces)  # if you prefer to work with numpy array for number crunching
textin_array = np.asarray(textin)
known_keys = np.asarray(keys)  # for fixed key, these keys are all the same

In [ ]:
from lascar import *
from lascar.tools.aes import sbox

def generate_selection_function(byte):
    def selection_with_guess(value, guess): # selection_with_guess function must take 2 arguments: value and guess
        return hamming(sbox[value[byte] ^ guess])
    return selection_with_guess

In [ ]:
mycontainer =  TraceBatchContainer(trace_array, textin_array)

In [ ]:
guess_range = range(256)

In [ ]:
cpa_engines = [CpaEngine("cpa_%02d" % i, generate_selection_function(i), guess_range) for i in range(16)]

In [ ]:
int(cpa_engines[0].name[-2:])

In [ ]:
help(cpa_engines[0])

In [ ]:
from lascar.output.parse_results import apply_parse
import pandas as pd
from IPython.display import clear_output
class JupyterOutputMethod(OutputMethod):
    def __init__(self, *engines):
        OutputMethod.__init__(self, engines)
        self.parsed_subkeys = 0
        self.results_list = [0] * 16
    
    def _update(self, engine, results):
        engine.output_parser_mode = "argmax"
        results_parsed = apply_parse(engine, abs(results))
        display(self.parsed_subkeys)
        if results_parsed is None:
            return
        idx = int(cpa_engine.name[-2:])
        self.results_list[idx] = results_parsed
        self.parsed_subkeys += 1
        if self.parsed_subkeys > 15:
            df = pd.DataFrame(self.results_list)
            fd = df.transpose()
            def formatter(stats):
                return str("{}".format(stats))
            #clear_output(wait=True)
            display(fd.head().style.format(formatter))
            self.parsed_subkeys = 0
        
    def _finalize(self):
        pass
        
session = Session(mycontainer, engines=cpa_engines, output_method=JupyterOutputMethod(*cpa_engines)).run(batch_size=50)

In [ ]:
x = [1, 2]
x[1] = "test"

In [ ]:
print(x)

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()
p = figure()
for i in range(16):
    results = cpa_engines[i].finalize()
    xrange = range(len(results[0x2B]))
    guess = abs(results).max(1).argmax()
    print("Best Guess is {:02X} (Corr = {})".format(guess, abs(results).max()))
    p.line(xrange, results[guess])
    
show(p)

In [ ]:
print(results)

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()
xrange = range(len(results[0x2B]))
print(xrange)
print(len(results))
p = figure()
#p.line(xrange, traces[5],line_color='red')
p.line(xrange, results[0x3C])
show(p)

In [ ]:
print(mycontainer.values)